Type Markdown and LaTeX:  𝛼2

In [ ]:
#Matplotlib Homework
%matplotlib notebook
%matplotlib inline
#Dependencies
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
import numpy as np
import scipy.stats as st
# Study data files
mouse_metadata = "data/Mouse_metadata.csv"
study_results = "data/Study_results.csv"
#Read in data files
mouse_metadata = pd.read_csv(mouse_metadata)
study_results = pd.read_csv(study_results)

# Observations
#There's a positive correlation between mouse weight and average tumor volume for mice on the Capomulin regimen. Ramicane seems to be slightly more promising than Capomulin at reducing tumor volume over time. Of the four drugs selected for comparison in the box plot only Capomulin and Ramicane performed better than the placebo.


# Combined Study Data

In [ ]:
#Merge tables with inner join
merged_table = pd.merge(mouse_metadata, study_results, on="Mouse ID")
del merged_table["Age_months"]
del merged_table["Metastatic Sites"]
merged_table

In [ ]:
#max_wt = merged_table["Weight (g)"].max()\nmax_wt
#min_wt = merged_table["Weight (g)"].min()\nmin_wt

In [ ]:
#Get array of unique Mouse IDs
unique_mice = merged_table["Mouse ID"].unique()
#unique_mice

In [ ]:
nbr_unique_mice = len(unique_mice)
print(f"There are {nbr_unique_mice} unique subjects.")

In [ ]:
rows_per_drug = merged_table["Drug Regimen"].value_counts()
rows_per_drug_df = rows_per_drug.to_frame() 
rows_per_drug_df = rows_per_drug_df.rename(columns={"Drug Regimen" : "Number of Data Pts."})
rows_per_drug_df.index.name = "Drug Name"
rows_per_drug_df

In [ ]:
tumor_vol = merged_table.groupby(["Drug Regimen"])
#Convert GroupBy Object to DataFrame
mean_tumor_vol_df = tumor_vol["Tumor Volume (mm3)"].mean().to_frame()
mean_tumor_vol_df.columns = ["Mean Tumor Vol. (mm3)"]
mean_tumor_vol_df

In [ ]:
median_tumor_vol_df = tumor_vol["Tumor Volume (mm3)"].median().to_frame()
median_tumor_vol_df.columns = ["Median Tumor Vol. (mm3)"]
median_tumor_vol_df

In [ ]:
var_tumor_vol_df = tumor_vol["Tumor Volume (mm3)"].var().to_frame()
var_tumor_vol_df.columns = ["Tumor Vol. Variance"]
var_tumor_vol_df

In [ ]:
sem_tumor_vol_df = tumor_vol["Tumor Volume (mm3)"].sem().to_frame()
sem_tumor_vol_df.columns = ["Tumor Vol. SEM"]
sem_tumor_vol_df

In [ ]:
std_tumor_vol_df = tumor_vol["Tumor Volume (mm3)"].std().to_frame()
std_tumor_vol_df.columns = ["Tumor Vol. STD"]
std_tumor_vol_df

In [ ]:
drugs = merged_table["Drug Regimen"].unique()
#drugs

In [ ]:
stats_merge1_df = pd.merge(mean_tumor_vol_df, median_tumor_vol_df, on="Drug Regimen")
#stats_merge1_df

In [ ]:
stats_merge2_df = pd.merge(stats_merge1_df, var_tumor_vol_df, on ="Drug Regimen")
#stats_merge2_df

In [ ]:
stats_merge3_df = pd.merge(stats_merge2_df, sem_tumor_vol_df, on="Drug Regimen")
#stats_merge3_df

# Summary Statistics Table

In [ ]:
full_stats_df = pd.merge(stats_merge3_df, std_tumor_vol_df, on="Drug Regimen")
full_stats_df.round(2)

# Tumor Volume Statistics by Drug

### Pandas Bar Chart

In [ ]:
%matplotlib inline
pandas_bar_plot = rows_per_drug_df[["Number of Data Pts."]]

pandas_bar_plot.plot(kind="bar", figsize=(5, 5), facecolor="blue")
plt.title("Data Points per Drug Regimen")
plt.tight_layout()
plt.ylabel("Number of Data Points")
plt.grid(linestyle="dotted")
plt.show()

### Pyplot Bar Chart

In [ ]:
#Bar plot using pyplot
drug = drugs
rows = rows_per_drug

x_axis = np.arange(0, 10, 1)
tick_locations = []
for x in x_axis:
    tick_locations.append(x)
    
plt.title("Data Points per Drug Regimen", fontsize=12)
plt.xlabel("Drug Name", fontsize=12)
plt.ylabel("Number of Data Points", fontsize=12)
plt.ylim(0, 240)

plt.bar(x_axis, rows, facecolor="blue", edgecolor="black", alpha=0.5, align="center",)
plt.xticks(tick_locations, drug, rotation="vertical")
plt.tight_layout()
plt.grid(linestyle="dotted")

plt.savefig("../../Images/DataPointsByDrug.png")
plt.show()

# Sex Distribution of Pymaceuticals Study Subjects

### Pyplot Pie Chart

In [ ]:
by_sex_df = merged_table[["Mouse ID", "Sex"]]
by_sex_df.set_index("Mouse ID")
new_data_df = by_sex_df.drop_duplicates()
new_data_df
count_df = new_data_df["Sex"].value_counts()
count_df

In [ ]:
counts = [125, 124]
labels = ["Male Mice", "Female Mice"]
colors = ["lightskyblue", "yellowgreen"]

plt.title("Sex Distribution of Subjects")
plt.pie(counts, labels = labels, colors = colors, autopct="%1.1f%%", shadow = "True", startangle = 90)
plt.axis("equal")
plt.savefig("../../Images/SexDistribution.png")
plt.show()

In [ ]:
count_df.plot(kind="pie", figsize=(5, 5), subplots=True, autopct="%1.1f%%", title="Sex Distribution of Subjects", labels=["Male Mice", "Female Mice"],
              legend=False, colors = ["lightskyblue", "yellowgreen"], startangle=90)

## Final tumor vol. of each mouse across : Capomulin, Ramicane, Infubinol, & Ceftamin

In [ ]:
Capomulin_table = merged_table.loc[(merged_table["Drug Regimen"]== "Capomulin")] 
Capomulin_table
#Capomulin_table["Mouse ID"].nunique()
Cap_avg_tum = Capomulin_table[["Mouse ID", "Timepoint", "Weight (g)", "Tumor Volume (mm3)"]]
mean_vols = Cap_avg_tum.groupby("Mouse ID").mean()
del mean_vols["Timepoint"]
Cap_mean_vols = mean_vols.rename(columns={"Tumor Volume (mm3)" : "Mean Tumor Vol"})
Cap_mean_vols["Weight (g)"] = Cap_mean_vols["Weight (g)"].astype('int')
Cap_mean_vols.round(1)

In [ ]:
Capomulin_data = Capomulin_table.loc[(Capomulin_table["Timepoint"]==45), :]
del Capomulin_data["Sex"]
del Capomulin_data["Drug Regimen"]
del Capomulin_data["Timepoint"]
Capomulin_data = Capomulin_data.set_index(["Mouse ID"])
Cap_final = Capomulin_data.rename(columns={"Tumor Volume (mm3)" : "Final Tumor Vol"})
Cap_final.round(1)

In [ ]:
Infubinol_table = merged_table.loc[(merged_table["Drug Regimen"]== "Infubinol")] 
Infubinol_table
#Infubinol_table["Mouse ID"].nunique()
Infubinol_data = Infubinol_table.loc[(Infubinol_table["Timepoint"]==45), :]
del Infubinol_data["Sex"]
del Infubinol_data["Drug Regimen"]
del Infubinol_data["Timepoint"]
Infubinol_data = Infubinol_data.set_index(["Mouse ID"])
Infubinol_data.round(1)
Inf_final = Infubinol_data.rename(columns={"Tumor Volume (mm3)" : "Final Tumor Vol"})
Inf_final.round(1)

In [ ]:
Ceftamin_table = merged_table.loc[(merged_table["Drug Regimen"]== "Ceftamin")] 
Ceftamin_table
#Ceftamin_table["Mouse ID"].nunique()
Ceftamin_data = Ceftamin_table.loc[(Ceftamin_table["Timepoint"]==45), :]
del Ceftamin_data["Sex"]
del Ceftamin_data["Drug Regimen"]
del Ceftamin_data["Timepoint"]
Ceftamin_data = Ceftamin_data.set_index(["Mouse ID"])
Ceftamin_data.round(1)
Cef_final = Ceftamin_data.rename(columns={"Tumor Volume (mm3)" : "Final Tumor Vol"})
Cef_final.round(1)

In [ ]:
Ramicane_table = merged_table.loc[(merged_table["Drug Regimen"] == "Ramicane")]
Ramicane_table
#Ramicane_table["Mouse ID"].nunique()
Ramicane_data = Ramicane_table.loc[(Ramicane_table["Timepoint"]==45), :]
del Ramicane_data["Sex"]
del Ramicane_data["Drug Regimen"]
del Ramicane_data["Timepoint"]
Ramicane_data = Ramicane_data.set_index(["Mouse ID"])
Ramicane_data.round(1)
Ram_final = Ramicane_data.rename(columns={"Tumor Volume (mm3)" : "Final Tumor Vol"})
Ram_final

# Box Plots

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest
Cap_volumes = Cap_final["Final Tumor Vol"]
quartiles = Cap_volumes.quantile([0.25, 0.5, 0.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq
print(f"The lower quartile of Capomulin volumes is: {round(lowerq)}")
print(f"The upper quartile of Capomulin volumes is: {round(upperq)}")
print(f"The IQR of Capomulin volumes is: {round(iqr)}; median is: {round(quartiles[0.5])} ")
lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values above {round(upper_bound)} and below {round(lower_bound)} could be outliers.")
print("++++++++++++++++++++++++++++++++++++++++")
Ram_volumes = Ram_final["Final Tumor Vol"]
quartiles = Ram_volumes.quantile([0.25, 0.5, 0.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq
print(f"The lower quartile of Ramicane volumes is: {round(lowerq)}")
print(f"The upper quartile of Ramicane volumes is: {round(upperq)}")
print(f"The IQR of Ramicane volumes is: {round(iqr)}; median is: {round(quartiles[0.5])} ")
lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values above {round(upper_bound)} and below {round(lower_bound)} could be outliers.")
print("++++++++++++++++++++++++++++++++++++++++")
Cef_volumes = Cef_final["Final Tumor Vol"]
quartiles = Cef_volumes.quantile([0.25, 0.5, 0.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq
print(f"The lower quartile of Ceftamin volumes is: {round(lowerq)}")
print(f"The upper quartile of Ceftamin volumes is: {round(upperq)}")
print(f"The IQR of Ceftamin volumes is: {round(iqr)}; median is: {round(quartiles[0.5])} ")
lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values above {round(upper_bound)} and below {round(lower_bound)} could be outliers.")
print("++++++++++++++++++++++++++++++++++++++++")
Inf_volumes = Inf_final["Final Tumor Vol"]
quartiles = Inf_volumes.quantile([0.25, 0.5, 0.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq
print(f"The lower quartile of Infubinol volumes is: {round(lowerq)}")
print(f"The upper quartile of Infubinol volumes is: {round(upperq)}")
print(f"The IQR of Infubinol volumes is: {round(iqr)}; median is: {round(quartiles[0.5])} ")
lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values above {round(upper_bound)} and below {round(lower_bound)} could be outliers.")
print("++++++++++++++++++++++++++++++++++++++++")
boxplot_data = [Cap_volumes, Ram_volumes, Cef_volumes, Inf_volumes]
fig = plt.figure(1, figsize=(10,10))
ax = fig.add_subplot()
ax.set_title("Final Tumor Volumes Across Top 4 Drugs", fontsize=16)
ax.set_xticklabels(["Capomulin", "Ramicane", "Ceftamin", "Infubinol"], fontsize=14)
ax.set_ylabel('Tumor Vol (mm3)', fontsize=16)
bp = ax.boxplot(boxplot_data, notch=True, showfliers=True)
for box in bp ["boxes"]:
    box.set(color = "green", linewidth=2)
for flier in bp ["fliers"]:
    flier.set(marker = 'o', color="black", alpha=0.5)
for whisker in bp["whiskers"]:
    whisker.set(color="blue", linewidth=2)
fig.savefig("../../Images/top4boxplt.png")

## Line and Scatter Plots

In [ ]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin
vol_by_timept = Capomulin_table.groupby("Timepoint").mean()
del vol_by_timept["Weight (g)"]
vol_by_timept = round((vol_by_timept), 1)
times = vol_by_timept.index
volumes = vol_by_timept["Tumor Volume (mm3)"]

plt.plot(times, volumes, linewidth=2)
plt.title("Time versus Tumor Vol for Capomulin")
volumes_handle, = plt.plot(times, volumes, color='red', label="Tumor Volume (mm3)")
plt.xlabel("Timepoint (days)", fontsize=12)
plt.ylabel("Tumor Volume (mm3)", fontsize=12)
plt.legend(loc="upper right")

plt.savefig("../../Images/TumorVol_over_Time_Capomulin.png")
plt.show()

In [ ]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen
x_axis = Cap_mean_vols["Weight (g)"]
y_axis = Cap_mean_vols["Mean Tumor Vol"]
mean_tumor_vol = Cap_mean_vols["Mean Tumor Vol"]
plt.title("Weight vs. Mean Tumor Vol for Capomulin", fontsize=13)
plt.xlabel("Mouse Weight (g)", fontsize=12)
plt.ylabel("Mean Tumor Volume (mm3)", fontsize=12)
plt.scatter(x_axis, y_axis, marker = "o", color = "blue")
plt.savefig("../../Images/Weight_v_MeanTumorVol.png")
weight = Cap_mean_vols["Weight (g)"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(weight, mean_tumor_vol)
regress_values = weight * slope + intercept
line_eq = "y= " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(weight,regress_values,"r-")
plt.annotate(line_eq,(20,36), fontsize=15, color="red")

In [ ]:
# Calculate the correlation coefficient and linear regression model for mouse weight and average tumor volume for the Capomulin regimen
weight = Cap_mean_vols["Weight (g)"]
mean_tumor_vol = Cap_mean_vols["Mean Tumor Vol"]
correlation = st.pearsonr(mean_tumor_vol, weight)
print(f"The correlation between mouse weight and mean tumor volume is {round(correlation[0],2)}")